In [3]:
import pandas as pd

# Carregando os dados
data_path = 'gpsMergeDataCleanerInter.csv'
data = pd.read_csv(data_path)

# Ajustando a leitura dos dados para o separador correto
data = pd.read_csv(data_path, sep=';')

# Verificando novamente as primeiras linhas para entender a estrutura dos dados
data.head()

,RTCData,RTCHora,GPSData,GPSHora,Lat,Long,Altgps,Vel,GPSUpdate,Ax,...,LatLogger,LongLogger,ElevationLogger,DataLogger,HoraLogger,SpeedLogger,SatellitesLogger,Seconds,LatInter,LongInter
0,2024-04-09,10:14:31,0/0/0,0:0:0,NaN,NaN,0.0,0.0,0,-0.08,...,-3.744695,-38.577413,17.009,2024-04-09,10:14:31,0.0,33,36871.0,-3.744695,-38.577413
1,2024-04-09,10:14:31,0/0/0,0:0:0,NaN,NaN,0.0,0.0,0,-0.08,...,-3.744695,-38.577413,17.009,2024-04-09,10:14:31,0.0,33,36871.0,-3.744696,-38.577413
2,2024-04-09,10:14:31,0/0/0,0:0:0,NaN,NaN,0.0,0.0,0,-0.08,...,-3.744695,-38.577413,17.009,2024-04-09,10:14:31,0.0,33,36871.0,-3.744696,-38.577413
3,2024-04-09,10:14:31,0/0/0,0:0:0,NaN,NaN,0.0,0.0,0,-0.08,...,-3.744695,-38.577413,17.009,2024-04-09,10:14:31,0.0,33,36871.0,-3.744696,-38.577413
4,2024-04-09,10:14:31,0/0/0,0:0:0,NaN,NaN,0.0,0.0,0,-0.08,...,-3.744695,-38.577413,17.009,2024-04-09,10:14:31,0.0,33,36871.0,-3.744696,-38.577413


In [4]:
# Aplicando o Filtro de Média Móvel (SMA) com janela de tamanho 5
sma_columns = ['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz']
for col in sma_columns:
    data[f'{col}_SMA'] = data[col].rolling(window=5).mean()

from scipy.signal import butter, filtfilt

# Função para aplicar o filtro passa-baixa Butterworth
def butter_lowpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs  # Frequência de Nyquist
    normal_cutoff = cutoff / nyq
    # Obter os coeficientes do filtro
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Definindo a frequência de corte e a frequência de amostragem
cutoff = 3.0  # Frequência de corte em Hz
fs = 10.0     # Frequência de amostragem em Hz, assumindo um valor comum

# Aplicando o filtro a cada coluna de interesse
for col in sma_columns:
    data[f'{col}_Butter'] = butter_lowpass_filter(data[col], cutoff, fs)


# Implementação simplificada do Filtro Complementar
# Este exemplo assume que os dados do acelerômetro e giroscópio já estão em um formato que possa ser combinado diretamente.
# O filtro complementar é frequentemente usado para estimar a orientação, combinando as forças gravitacionais (acelerômetro)
# e a taxa de rotação (giroscópio).

# Parâmetros do filtro
alpha = 0.98  # Coeficiente do filtro, determina o "peso" da estimativa do giroscópio vs. acelerômetro

# Inicialização de colunas para os resultados do filtro complementar
for col in ['Ax', 'Ay', 'Az']:
    data[f'{col}_Comp'] = 0.0
for col in ['Gx', 'Gy', 'Gz']:
    data[f'{col}_Comp'] = 0.0

# Aplicando o filtro complementar
# Nota: Esta é uma simplificação e não representa uma implementação real de fusão de sensores.
# Normalmente, o filtro complementar requer integração do giroscópio e diferenciação ou integração do acelerômetro.
# Estamos apenas misturando os sinais com um fator fixo como uma demonstração simplificada.
for col in sma_columns:
    if col.startswith('A'):  # Para o acelerômetro, usamos 1 - alpha
        data[f'{col}_Comp'] = (1 - alpha) * data[col]
    else:  # Para o giroscópio, usamos alpha
        data[f'{col}_Comp'] = alpha * data[col]

# Salvando os resultados em um novo arquivo CSV
comp_filtered_data_path = 'sma_Butterworth_comp_filtered_data.csv'
data.to_csv(comp_filtered_data_path, index=False)

comp_filtered_data_path

'sma_Butterworth_comp_filtered_data.csv'